# Book Recommendation System

### Manuel Iglesias

1. Read in Packages and Data

In [ ]:
!pip install langdetect

# libraries and packages
import pandas as pd
import numpy as np
import re

from langdetect import detect

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from nltk import punkt


from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from scipy.spatial import distance
from scipy.sparse import hstack


In [ ]:
#from google.colab import files
#upload = files.upload()


In [ ]:
data= pd.read_csv('book_data.csv')
# read in file
#file = 'book_data.csv'
#data = pd.read_csv(file)

In [ ]:
#show file
data.head()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous. Losing means cer...,NaN,Hardcover,9.78044E+12,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,US Edition,Paperback,9.78044E+12,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...
2,Harper Lee,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9.78006E+12,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,«È cosa ormai risaputa che a uno scapolo in po...,"Modern Library Classics, USA / CAN",Paperback,9.78068E+12,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...
4,Stephenie Meyer,About three things I was absolutely positive.F...,NaN,Paperback,9.78032E+12,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...


In [ ]:
# for usability and lack of processing power, shorten the dataset
df = data.head(30000)

2. Clean up data and remove unnecessary columns

In [ ]:
# convert to strings from series
df['book_desc'] = df['book_desc'].astype(str)
df['book_title'] = df['book_title'].astype(str)
df['genres'] = df['genres'].astype(str)

# page numbers to numeric
df['book_pages'] = df['book_pages'].str.extract('(\d+)').astype(float)

#drop image URL, edition, format, isbn
df = df.drop(columns=['image_url', 'book_edition', 'book_format', 'book_isbn'])

#separage out generes
df['genres'] = df['genres'].str.replace('|', ', ')

# drop NAs
df.dropna(inplace=True)

/var/folders/4m/2bfrh3v504d_kr830y5f840w0000gn/T/ipykernel_11472/3680474687.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['book_desc'] = df['book_desc'].astype(str)
/var/folders/4m/2bfrh3v504d_kr830y5f840w0000gn/T/ipykernel_11472/3680474687.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['book_title'] = df['book_title'].astype(str)
/var/folders/4m/2bfrh3v504d_kr830y5f840w0000gn/T/ipykernel_11472/3680474687.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [ ]:
# drop duplicate titles (keep first)
df.drop_duplicates(subset='book_title', keep='first', inplace=True)
df.head(10)

,book_authors,book_desc,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres
0,Suzanne Collins,Winning will make you famous. Losing means cer...,374.0,4.33,5519135,160706,The Hunger Games,"Young Adult, Fiction, Science Fiction, Dystopi..."
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,870.0,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,"Fantasy, Young Adult, Fiction"
2,Harper Lee,The unforgettable novel of a childhood in a sl...,324.0,4.27,3745197,79450,To Kill a Mockingbird,"Classics, Fiction, Historical, Historical Fict..."
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,«È cosa ormai risaputa che a uno scapolo in po...,279.0,4.25,2453620,54322,Pride and Prejudice,"Classics, Fiction, Romance"
4,Stephenie Meyer,About three things I was absolutely positive.F...,498.0,3.58,4281268,97991,Twilight,"Young Adult, Fantasy, Romance, Paranormal, Vam..."
5,Markus Zusak,Trying to make sense of the horrors of World W...,552.0,4.36,1485632,100821,The Book Thief,"Historical, Historical Fiction, Fiction, Young..."
6,C.S. Lewis|Pauline Baynes,"Journeys to the end of the world, fantastic cr...",767.0,4.25,437829,9439,The Chronicles of Narnia,"Fantasy, Classics, Fiction, Young Adult, Child..."
7,George Orwell,مزرعة الحيوانات هي رائعة جورج أورويل الخالدة.....,122.0,3.90,2235084,42156,Animal Farm,"Classics, Fiction, Science Fiction, Dystopia, ..."
8,Margaret Mitchell,Gone with the Wind is a novel written by Marga...,1037.0,4.29,969181,17452,Gone with the Wind,"Classics, Historical, Historical Fiction, Fict..."
9,J.R.R. Tolkien,لجزء الثالث من ملحمة جيه أر أر تولكين الرائعة ...,1728.0,4.59,99793,1652,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,"Fantasy, Fiction, Classics"


In [ ]:
def safe_detect(text):
    try:
        return detect(text)
    except:
        return "unknown"  # Return 'unknown' or any other default you find suitable

# Apply safe language detection
df['detect'] = df['book_desc'].apply(lambda x: safe_detect(x) if x.strip() != "" else "en")

# Filter out non-English descriptions
df = df[df['detect'] == 'en']

# Optionally, remove the 'detect' column if it's no longer needed
df.drop(columns=['detect'], inplace=True)

In [ ]:
unique = df['book_title'].value_counts()
unique

The Hunger Games                 1
Two Witnesses                    1
After Midnight                   1
The Rose Knight                  1
I Loved You More                 1
                                ..
Goodbye Tsugumi                  1
The Women                        1
The Book of Human Skin           1
The Hypnotist                    1
Parcival: Ridder van de Graal    1
Name: book_title, Length: 22717, dtype: int64

In [ ]:
unique_titles_count = df['book_title'].nunique()
print("Total number of unique titles:", unique_titles_count)

Total number of unique titles: 22717


In [ ]:
df.head(10)

,book_authors,book_desc,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres
0,Suzanne Collins,Winning will make you famous. Losing means cer...,374.0,4.33,5519135,160706,The Hunger Games,"Young Adult, Fiction, Science Fiction, Dystopi..."
1,J.K. Rowling|Mary GrandPré,There is a door at the end of a silent corrido...,870.0,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,"Fantasy, Young Adult, Fiction"
2,Harper Lee,The unforgettable novel of a childhood in a sl...,324.0,4.27,3745197,79450,To Kill a Mockingbird,"Classics, Fiction, Historical, Historical Fict..."
4,Stephenie Meyer,About three things I was absolutely positive.F...,498.0,3.58,4281268,97991,Twilight,"Young Adult, Fantasy, Romance, Paranormal, Vam..."
5,Markus Zusak,Trying to make sense of the horrors of World W...,552.0,4.36,1485632,100821,The Book Thief,"Historical, Historical Fiction, Fiction, Young..."
6,C.S. Lewis|Pauline Baynes,"Journeys to the end of the world, fantastic cr...",767.0,4.25,437829,9439,The Chronicles of Narnia,"Fantasy, Classics, Fiction, Young Adult, Child..."
8,Margaret Mitchell,Gone with the Wind is a novel written by Marga...,1037.0,4.29,969181,17452,Gone with the Wind,"Classics, Historical, Historical Fiction, Fict..."
10,John Green,Despite the tumor-shrinking medical miracle th...,313.0,4.24,2881648,147270,The Fault in Our Stars,"Young Adult, Fiction, Romance, Contemporary"
11,Douglas Adams,Seconds before the Earth is demolished to make...,193.0,4.21,1155911,23919,The Hitchhiker's Guide to the Galaxy,"Science Fiction, Fiction, Humor, Fantasy, Clas..."
12,Shel Silverstein,"""Once there was a tree...and she loved a littl...",64.0,4.37,789681,15694,The Giving Tree,"Childrens, Childrens, Picture Books, Classics,..."


3. Join text columns for analysis

In [ ]:
# join text columns for analysis
df['features'] = df['book_desc'] + ' ' + df['book_authors'] + ' ' + df['genres']

4. Preprocess

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
# preprocess data
def preprocess_feature(features):
    # Remove non-alphanumeric characters
    cleaned_text = re.sub(r'\W+', ' ', features).lower()

    # Tokenize
    tokens = word_tokenize(cleaned_text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Stemming and Lemmatization
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    stemmed = [stemmer.stem(word) for word in tokens]
    lemmatized = [lemmatizer.lemmatize(word) for word in stemmed]

    # Reconstruct the message from lemmatized tokens
    return ' '.join(lemmatized)

# Apply preprocessing to the features column
df['features'] = df['features'].apply(preprocess_feature)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/bridgetvaughey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bridgetvaughey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bridgetvaughey/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# normalize numeric values
scaler = MinMaxScaler()
book_pages_scaled = scaler.fit_transform(df[['book_pages']])
book_rating_scaled = scaler.fit_transform(df[['book_rating']])
book_rating_count_scaled = scaler.fit_transform(df[['book_rating_count']])
book_review_count_scaled = scaler.fit_transform(df[['book_review_count']])


5. Apply tfidf (from asn 3) and Concatenate Features with Normalized Numeric Values

In [ ]:
# apply tfidf vectorizer object with parameters
tfidf_vectorizer = TfidfVectorizer(min_df=1, norm='l2', smooth_idf=True, use_idf=True, ngram_range=(1, 1))

# fit on data and transform
tfidf_matrix = tfidf_vectorizer.fit_transform(df['features'])

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Apply LSA
n_components = 50  # You can adjust the number of components
lsa = TruncatedSVD(n_components=n_components, random_state=42)
tfidf_lsa = lsa.fit_transform(tfidf_matrix)

In [ ]:
# Combine tfidf_lsa and numerics
combined_features = np.concatenate((tfidf_lsa, book_pages_scaled, book_rating_scaled, book_rating_count_scaled, book_review_count_scaled), axis=1)



6. Create Cosine Similarity Matrix (from similarity lab)

In [ ]:
# Make tfidf_matrix term-document matrix after applying tfidf
cosine_sim_matrix = cosine_similarity(combined_features)
# make dataframe
cosine_similarity_df = pd.DataFrame(cosine_sim_matrix, index=df['book_title'], columns=df['book_title'])

In [ ]:
cosine_similarity_df.head(10)

book_title,The Hunger Games,Harry Potter and the Order of the Phoenix,To Kill a Mockingbird,Twilight,The Book Thief,The Chronicles of Narnia,Gone with the Wind,The Fault in Our Stars,The Hitchhiker's Guide to the Galaxy,The Giving Tree,...,"Dear Bridget, I Want You",The Clockwork Dynasty,Red Dragon:,The Trouble With Being a Duke,Louie the Llama Discovers Christmas,You Are a Badass: How to Stop Doubting Your Greatness and Start Living an Awesome Life,Danny: The Champion of the World,"Sex Criminals, Vol. 1: One Weird Trick",The Creepy Case Files of Margo Maloo,Parcival: Ridder van de Graal
book_title,,,,,,,,,,,,,,,,,,,,,
The Hunger Games,1.000000,0.802136,0.931796,0.936174,0.845972,0.579381,0.656026,0.955523,0.732517,0.653100,...,0.497764,0.498330,0.553626,0.467216,0.510190,0.495219,0.526133,0.471831,0.485860,0.501352
Harry Potter and the Order of the Phoenix,0.802136,1.000000,0.893784,0.820182,0.854885,0.888568,0.890608,0.832651,0.955187,0.921542,...,0.856224,0.824812,0.887507,0.801117,0.867222,0.825078,0.887516,0.792590,0.839655,0.846734
To Kill a Mockingbird,0.931796,0.893784,1.000000,0.928313,0.888616,0.750853,0.838280,0.926610,0.858010,0.814055,...,0.675152,0.667725,0.723911,0.650058,0.683738,0.671062,0.693329,0.625952,0.646093,0.688588
Twilight,0.936174,0.820182,0.928313,1.000000,0.817228,0.628872,0.690398,0.907677,0.749875,0.692174,...,0.551383,0.529295,0.597461,0.518607,0.555497,0.553928,0.561415,0.508199,0.530602,0.541537
The Book Thief,0.845972,0.854885,0.888616,0.817228,1.000000,0.764119,0.860092,0.909225,0.826378,0.816582,...,0.708192,0.723639,0.753163,0.724054,0.724798,0.705017,0.739762,0.651133,0.690211,0.724832
The Chronicles of Narnia,0.579381,0.888568,0.750853,0.628872,0.764119,1.000000,0.889056,0.666447,0.915962,0.926455,...,0.881639,0.882271,0.904814,0.832572,0.915286,0.846798,0.925861,0.811304,0.864982,0.911351
Gone with the Wind,0.656026,0.890608,0.838280,0.690398,0.860092,0.889056,1.000000,0.725482,0.918032,0.920639,...,0.864910,0.880176,0.892605,0.908111,0.890130,0.830709,0.883634,0.798183,0.840257,0.892689
The Fault in Our Stars,0.955523,0.832651,0.926610,0.907677,0.909225,0.666447,0.725482,1.000000,0.792194,0.732434,...,0.611336,0.581933,0.643891,0.566495,0.614636,0.605318,0.629165,0.559373,0.572846,0.599734
The Hitchhiker's Guide to the Galaxy,0.732517,0.955187,0.858010,0.749875,0.826378,0.915962,0.918032,0.792194,1.000000,0.944104,...,0.896845,0.883293,0.923572,0.837004,0.921098,0.859285,0.929643,0.821501,0.863348,0.900577


7. Define Recommendation Function

In [ ]:
def recommended_books(book_title, similarity_df, top_n=10):
    if book_title in similarity_df.index:
        # similarity scores for book
        sim_scores = similarity_df.loc[book_title]

        # sort the books based on similarity scores (descending order)
        top_books = sim_scores.sort_values(ascending=False).head(top_n + 1)[1:]

        # DataFrame for top_n similar books
        recommended_books = pd.DataFrame(top_books).reset_index()
        recommended_books.columns = ['Book Title', 'Similarity Score']

        # Merge with original book Data
        recommended_books = recommended_books.merge(df, left_on='Book Title', right_on='book_title', how='left')
        recommended_books = recommended_books[['Book Title', 'Similarity Score', 'book_authors', 'genres', 'book_desc', 'book_pages', 'book_rating']]


        return recommended_books
    else:
        return None

8. Test Function

In [ ]:
recommended_books = recommend_books('The Hunger Games', cosine_similarity_df)
recommended_books

,Book Title,Similarity Score,book_authors,genres,book_desc,book_pages,book_rating
0,The Fault in Our Stars,0.955523,John Green,"Young Adult, Fiction, Romance, Contemporary",Despite the tumor-shrinking medical miracle th...,313.0,4.24
1,Divergent,0.942101,Veronica Roth,"Young Adult, Science Fiction, Dystopia, Fictio...","In Beatrice Prior's dystopian Chicago world, s...",487.0,4.22
2,Harry Potter and the Sorcerer's Stone,0.938962,J.K. Rowling|Mary GrandPré,"Fantasy, Young Adult, Fiction",Harry Potter's life is miserable. His parents ...,320.0,4.46
3,Twilight,0.936174,Stephenie Meyer,"Young Adult, Fantasy, Romance, Paranormal, Vam...",About three things I was absolutely positive.F...,498.0,3.58
4,Mockingjay,0.935855,Suzanne Collins,"Young Adult, Science Fiction, Dystopia, Fictio...",My name is Katniss Everdeen.Why am I not dead?...,392.0,4.03
5,To Kill a Mockingbird,0.931796,Harper Lee,"Classics, Fiction, Historical, Historical Fict...",The unforgettable novel of a childhood in a sl...,324.0,4.27
6,Catching Fire,0.926937,Suzanne Collins,"Young Adult, Science Fiction, Dystopia, Fictio...",Sparks are igniting.Flames are spreading.And t...,391.0,4.29
7,Gone Girl,0.918955,Gillian Flynn,"Fiction, Mystery, Thriller, Mystery, Crime","On a warm summer morning in North Carthage, Mi...",415.0,4.05
8,Nineteen Eighty-Four,0.890731,George Orwell|Thomas Pynchon,"Classics, Fiction, Science Fiction, Science Fi...",Alternative cover edition can be found here.NO...,339.0,4.16
9,The Girl with the Dragon Tattoo,0.886953,Stieg Larsson|Reg Keeland,"Fiction, Mystery, Thriller, Mystery, Crime","A spellbinding amalgam of murder mystery, fami...",465.0,4.13
